# Bike sharing in Washington
<em>Individual Project Python<em>
___

                              Master in Business Analytics & Big data
                                 Professor Juan Luis Cano Rodriguez
                                           Martin Hofbauer
                                              _________

## Libraries

In [1]:
import dask
import dask.dataframe as dd
import dask.array as da
import numpy as np
import dask_ml
from dask_ml.preprocessing import Categorizer, DummyEncoder
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline

C:\Users\Martin\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


## Data Loading

In [2]:
hour_data = dd.read_csv(
    "https://gist.githubusercontent.com/geraldwal/b5a83f4c670abe0a662abce558e5d433/raw/bce4bbfc63355606e4503964e25798b5d2190b9b/hour%2520-%2520Python%2520Bike%2520Sharing",
    sep=",",
    parse_dates=["dteday"],
)

In [3]:
hour_data.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 17 entries, instant to cnt
dtypes: datetime64[ns](1), float64(4), int64(12)

## Preparation for Baseline

### Renaming the Colums

In [4]:
new_columns = [
    "instant",
    "dteday",
    "season",
    "year",
    "month",
    "hour",
    "holiday",
    "weekday",
    "workingday",
    "weather",
    "temp",
    "atemp",
    "humidity",
    "windspeed",
    "casual",
    "registered",
    "count",
]
hour_data= hour_data.rename(columns=dict(zip(hour_data.columns, new_columns)))

### Applying the Corrrect Datatypes and DummyEncoding

In [5]:
categs = ['season', 'year', 'month', 'hour', 'holiday', 'weekday', 'workingday', 'weather']

In [6]:
pipeline = make_pipeline(Categorizer(columns=categs), DummyEncoder(columns=categs))
hour_data_cl_hot_encoded = pipeline.fit_transform(hour_data)

### Splitting the Dataset

In [7]:
def split_data(dataset, Target):
    X = dataset.loc[:, dataset.columns != Target]
    y = dataset.loc[:, Target]
    train_size = int(len(dataset) * 0.875)
    X_train, X_test, y_train, y_test = (
        X[0:train_size],
        X[train_size : len(dataset)],
        y[0:train_size],
        y[train_size : len(dataset)],
    )
    return X_train, X_test, y_train, y_test

In [8]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = split_data(hour_data_cl_hot_encoded, "registered")
x_train_casual, x_test_casual, y_train_casual, y_test_casual = split_data(hour_data_cl_hot_encoded, "casual")
x_train_count, x_test_count, y_train_count, y_test_count = split_data(hour_data_cl_hot_encoded, "count")

NotImplementedError: 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.